In [1]:
import pandas as pd
import numpy as np
import random
import os
import sys
from math import sqrt
sys.path.append('../..')
from modules import utils
import tensorflow as tf
import tensorflow_probability as tfp
tfk = tf.keras
tf.keras.backend.set_floatx('float64')
tfd = tfp.distributions
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)

In [2]:
neg_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)

#### The data

In [3]:
jinja_df = pd.read_csv('../data/jinja_data.csv', parse_dates=['timestamp'])
jinja_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
1,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
2,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
3,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
4,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23


In [4]:
latitudes = jinja_df['latitude'].unique()
longitudes = jinja_df['longitude'].unique()
device_ids = jinja_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(10, 10, 10)

In [5]:
final_df = pd.DataFrame()
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value']
for i, device_id in enumerate(device_ids):
    device_df = utils.get_device_data(jinja_df, device_id, cols)
    processed_df = utils.preprocessing(device_df)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5
0,452909.0,0.437337,33.211051,12.2844
1,452910.0,0.437337,33.211051,11.6507
2,452911.0,0.437337,33.211051,22.3980
3,452912.0,0.437337,33.211051,17.4937
4,452913.0,0.437337,33.211051,25.1622


#### Model training and validation

In [6]:
def bnn(X_train, y_train, epochs=1000, optimizer='RMSProp'):
#     model = tf.keras.Sequential([
#     tfp.layers.DenseVariational(units=128, make_prior_fn=tfp.layers.default_multivariate_normal_fn, 
#                                 make_posterior_fn=tfp.layers.default_multivariate_normal_fn),
#     tfp.layers.DenseVariational(units=64, make_prior_fn=tfp.layers.default_multivariate_normal_fn,
#                                make_posterior_fn=tfp.layers.default_multivariate_normal_fn),
#     tfp.layers.DenseVariational(units=1, make_prior_fn=tfp.layers.default_multivariate_normal_fn,
#                                make_posterior_fn=tfp.layers.default_multivariate_normal_fn),
#     ])
    
# #     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.MeanSquaredError())
#     model.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
#     model.fit(X_train, y_train, batch_size=32, epochs=epochs, callbacks=None, validation_split=0.2)
    prior = tfd.Independent(tfd.Normal(loc=tf.zeros(1, dtype=tf.float64), scale=1.0), 
                            reinterpreted_batch_ndims=1)
    model = tfk.Sequential([
        tfk.layers.InputLayer(input_shape=(3,), name='input'),
        tfk.layers.Dense(10, activation='relu', name='dense_1'),
        tfk.layers.Dense(tfp.layers.MultivariateNormalTriL.params_size(1), activation=None, name='distribution_weights'),
        tfp.layers.MultivariateNormalTriL(1, activity_regularizer=tfp.layers.KLDivergenceRegularizer(prior, weight=1/32), 
                                          name='output')], name='model')
    
#     model.compile(optimizer=optimizer, loss=neg_log_likelihood)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
    model.fit(X_train, y_train, batch_size=32, epochs=epochs, callbacks=None, validation_split=0.2)
    return model

In [7]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
    assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
    assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
    assert(len(train_df.longitude.unique()) == len(longitudes)-1)
    assert len(final_df) == len(test_df) + len(train_df)
    
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
#     data_train =tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))
#     data_train = data_train.batch(32).repeat(n_epochs)
    X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
#     data_test =tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values))
#     data_test = data_test.batch(1)
    X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
    
    model = bnn(X_train, y_train)
    y_pred = model.predict(X_test)
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mape

In [8]:
rmse_list, mape_list = [], []
for i in range(len(latitudes)):
    rmse, mape = cross_validation(final_df, i)
    rmse_list.append(rmse)
    mape_list.append(mape)
    print(f'{device_ids[i]} successful')

Epoch 1/500
294/294 [==============================] - 3s 4ms/step - loss: 6967399892.2165 - val_loss: 429886.4483
Epoch 2/500
294/294 [==============================] - 1s 2ms/step - loss: 368683.3914 - val_loss: 395774.1543
Epoch 3/500
294/294 [==============================] - 1s 2ms/step - loss: 367137.4854 - val_loss: 356468.4701
Epoch 4/500
294/294 [==============================] - 1s 2ms/step - loss: 366709.4671 - val_loss: 365744.0095
Epoch 5/500
294/294 [==============================] - 1s 2ms/step - loss: 365021.8859 - val_loss: 366527.4536
Epoch 6/500
294/294 [==============================] - 1s 2ms/step - loss: 364809.5418 - val_loss: 365357.1962
Epoch 7/500
294/294 [==============================] - 1s 2ms/step - loss: 363092.4168 - val_loss: 353530.9022
Epoch 8/500
294/294 [==============================] - 1s 2ms/step - loss: 361656.7071 - val_loss: 319895.9725
Epoch 9/500
294/294 [==============================] - 1s 2ms/step - loss: 360849.5516 - val_loss: 393027.55

Epoch 147/500
294/294 [==============================] - 1s 3ms/step - loss: 209327.1345 - val_loss: 209988.5109
Epoch 148/500
294/294 [==============================] - 1s 3ms/step - loss: 208956.9304 - val_loss: 187486.0133
Epoch 149/500
294/294 [==============================] - 1s 2ms/step - loss: 207466.3718 - val_loss: 226227.0633
Epoch 150/500
294/294 [==============================] - 1s 2ms/step - loss: 206349.7853 - val_loss: 202261.4390
Epoch 151/500
294/294 [==============================] - 1s 2ms/step - loss: 205340.5622 - val_loss: 180627.2839
Epoch 152/500
294/294 [==============================] - 1s 2ms/step - loss: 204432.0006 - val_loss: 184442.6980
Epoch 153/500
294/294 [==============================] - 1s 2ms/step - loss: 203530.9849 - val_loss: 216617.3356
Epoch 154/500
294/294 [==============================] - 1s 2ms/step - loss: 201491.9985 - val_loss: 212295.7809
Epoch 155/500
294/294 [==============================] - 1s 2ms/step - loss: 200900.2197 - val_l

294/294 [==============================] - 1s 2ms/step - loss: 34665.6957 - val_loss: 24329.1750
Epoch 293/500
294/294 [==============================] - 1s 2ms/step - loss: 33953.7922 - val_loss: 28898.7201
Epoch 294/500
294/294 [==============================] - 1s 2ms/step - loss: 33531.5974 - val_loss: 33056.2256
Epoch 295/500
294/294 [==============================] - 1s 2ms/step - loss: 32850.5529 - val_loss: 21843.0233
Epoch 296/500
294/294 [==============================] - 1s 2ms/step - loss: 31954.4207 - val_loss: 45460.2406
Epoch 297/500
294/294 [==============================] - 1s 2ms/step - loss: 31597.8961 - val_loss: 26084.3874
Epoch 298/500
294/294 [==============================] - 1s 3ms/step - loss: 31065.0567 - val_loss: 36250.8623
Epoch 299/500
294/294 [==============================] - 1s 3ms/step - loss: 30471.7632 - val_loss: 32762.3444
Epoch 300/500
294/294 [==============================] - 1s 2ms/step - loss: 29607.1743 - val_loss: 34900.1975
Epoch 301/500
2

294/294 [==============================] - 1s 2ms/step - loss: 446.6954 - val_loss: 655.9194
Epoch 443/500
294/294 [==============================] - 1s 2ms/step - loss: 446.2879 - val_loss: 752.3831
Epoch 444/500
294/294 [==============================] - 1s 2ms/step - loss: 446.0112 - val_loss: 750.4677
Epoch 445/500
294/294 [==============================] - 1s 2ms/step - loss: 447.5573 - val_loss: 783.4316
Epoch 446/500
294/294 [==============================] - 1s 2ms/step - loss: 449.7531 - val_loss: 808.3596
Epoch 447/500
294/294 [==============================] - 1s 2ms/step - loss: 445.6591 - val_loss: 665.1744
Epoch 448/500
294/294 [==============================] - 1s 2ms/step - loss: 447.0376 - val_loss: 789.4854
Epoch 449/500
294/294 [==============================] - 1s 2ms/step - loss: 447.8012 - val_loss: 672.3408
Epoch 450/500
294/294 [==============================] - 1s 2ms/step - loss: 446.8754 - val_loss: 655.5924
Epoch 451/500
294/294 [============================

325/325 [==============================] - 1s 2ms/step - loss: 170904.4803 - val_loss: 179695.6504
Epoch 91/500
325/325 [==============================] - 1s 2ms/step - loss: 170197.1239 - val_loss: 166576.5082
Epoch 92/500
325/325 [==============================] - 1s 2ms/step - loss: 169328.4156 - val_loss: 164297.1684
Epoch 93/500
325/325 [==============================] - 1s 2ms/step - loss: 168396.2573 - val_loss: 162054.6286
Epoch 94/500
325/325 [==============================] - 1s 2ms/step - loss: 167348.9772 - val_loss: 170970.7516
Epoch 95/500
325/325 [==============================] - 1s 2ms/step - loss: 166744.9602 - val_loss: 171890.4267
Epoch 96/500
325/325 [==============================] - 1s 2ms/step - loss: 165860.9691 - val_loss: 195115.3597
Epoch 97/500
325/325 [==============================] - 1s 2ms/step - loss: 164855.7284 - val_loss: 159923.8317
Epoch 98/500
325/325 [==============================] - 1s 2ms/step - loss: 164134.9178 - val_loss: 161690.3083
Epoch

...

Epoch 49/500
320/320 [==============================] - 1s 2ms/step - loss: 374355.1376 - val_loss: 403233.1243
Epoch 50/500
320/320 [==============================] - 1s 2ms/step - loss: 372631.6355 - val_loss: 346968.6235
Epoch 51/500
320/320 [==============================] - 1s 2ms/step - loss: 370536.7744 - val_loss: 365927.7853
Epoch 52/500
320/320 [==============================] - 1s 2ms/step - loss: 368685.4079 - val_loss: 334227.0597
Epoch 53/500
320/320 [==============================] - 1s 2ms/step - loss: 366744.2221 - val_loss: 345358.1901
Epoch 54/500
320/320 [==============================] - 1s 2ms/step - loss: 365062.8759 - val_loss: 346079.5918
Epoch 55/500
320/320 [==============================] - 1s 2ms/step - loss: 362765.1539 - val_loss: 356962.4290
Epoch 56/500
320/320 [==============================] - 1s 2ms/step - loss: 360367.1654 - val_loss: 355693.2078
Epoch 57/500
320/320 [==============================] - 1s 2ms/step - loss: 359039.8467 - val_loss: 3029

Epoch 122/500
320/320 [==============================] - 1s 2ms/step - loss: 224188.5068 - val_loss: 245265.7680
Epoch 123/500
320/320 [==============================] - 1s 2ms/step - loss: 222665.4731 - val_loss: 240085.8148
Epoch 124/500
320/320 [==============================] - 1s 2ms/step - loss: 220048.7206 - val_loss: 208282.8234
Epoch 125/500
320/320 [==============================] - 1s 2ms/step - loss: 217462.4660 - val_loss: 228590.9874
Epoch 126/500
320/320 [==============================] - 1s 2ms/step - loss: 215829.5932 - val_loss: 202126.9170
Epoch 127/500
320/320 [==============================] - 1s 2ms/step - loss: 212958.5701 - val_loss: 219683.3810
Epoch 128/500
320/320 [==============================] - 1s 2ms/step - loss: 211134.3109 - val_loss: 233679.4784
Epoch 129/500
320/320 [==============================] - 1s 2ms/step - loss: 208602.2782 - val_loss: 195068.9595
Epoch 130/500
320/320 [==============================] - 1s 2ms/step - loss: 206706.2436 - val_l

Epoch 195/500
320/320 [==============================] - 1s 2ms/step - loss: 72498.9694 - val_loss: 69361.4117
Epoch 196/500
320/320 [==============================] - 1s 2ms/step - loss: 69755.4277 - val_loss: 59521.6755
Epoch 197/500
320/320 [==============================] - 1s 2ms/step - loss: 70916.9708 - val_loss: 63365.7342
Epoch 198/500
320/320 [==============================] - 1s 2ms/step - loss: 66247.5316 - val_loss: 53677.1764
Epoch 199/500
320/320 [==============================] - 1s 2ms/step - loss: 61473.6669 - val_loss: 32624.2926
Epoch 200/500
320/320 [==============================] - 1s 2ms/step - loss: 60156.9062 - val_loss: 66555.7661
Epoch 201/500
320/320 [==============================] - 1s 2ms/step - loss: 59633.5976 - val_loss: 55018.4368
Epoch 202/500
320/320 [==============================] - 1s 2ms/step - loss: 58871.3429 - val_loss: 56907.0547
Epoch 203/500
320/320 [==============================] - 1s 2ms/step - loss: 58296.9792 - val_loss: 56826.0216
E

Epoch 269/500
320/320 [==============================] - 1s 2ms/step - loss: 14655.4950 - val_loss: 12634.9809
Epoch 270/500
320/320 [==============================] - 1s 2ms/step - loss: 14191.7209 - val_loss: 18169.9641
Epoch 271/500
320/320 [==============================] - 1s 2ms/step - loss: 13908.0915 - val_loss: 11612.9946
Epoch 272/500
320/320 [==============================] - 1s 2ms/step - loss: 13385.8691 - val_loss: 6503.2506
Epoch 273/500
320/320 [==============================] - 1s 2ms/step - loss: 12958.4698 - val_loss: 18782.5285
Epoch 274/500
320/320 [==============================] - 1s 2ms/step - loss: 12670.3562 - val_loss: 11032.9387
Epoch 275/500
320/320 [==============================] - 1s 2ms/step - loss: 12128.2353 - val_loss: 10502.5611
Epoch 276/500
320/320 [==============================] - 1s 2ms/step - loss: 11724.0937 - val_loss: 13832.0610
Epoch 277/500
320/320 [==============================] - 1s 2ms/step - loss: 11359.6407 - val_loss: 6066.4430
Epo

320/320 [==============================] - 1s 2ms/step - loss: 412.3776 - val_loss: 431.4612
Epoch 421/500
320/320 [==============================] - 1s 2ms/step - loss: 413.7087 - val_loss: 457.5685
Epoch 422/500
320/320 [==============================] - 1s 2ms/step - loss: 411.3549 - val_loss: 415.8944
Epoch 423/500
320/320 [==============================] - 1s 3ms/step - loss: 411.1729 - val_loss: 422.7001
Epoch 424/500
320/320 [==============================] - 1s 2ms/step - loss: 411.2472 - val_loss: 409.6256
Epoch 425/500
320/320 [==============================] - 1s 2ms/step - loss: 412.2683 - val_loss: 425.6915
Epoch 426/500
320/320 [==============================] - 1s 2ms/step - loss: 413.3239 - val_loss: 438.4403
Epoch 427/500
320/320 [==============================] - 1s 2ms/step - loss: 412.8761 - val_loss: 412.4759
Epoch 428/500
320/320 [==============================] - 1s 2ms/step - loss: 410.4463 - val_loss: 429.6137
Epoch 429/500
320/320 [============================

In [9]:
mean_rmse = np.mean(rmse_list)          
mean_rmse

116.00910040304066

In [ ]:
rmse_list

In [ ]:
mean_mape = np.mean(mape_list)
mean_mape

In [ ]:
mape_list